

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_PL.ipynb)




# **Detect entities in Polish text**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

## 2. Start the Spark session

In [8]:
import pandas as pd
import numpy as np
import json

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 4.2.8


## 3. Select the DL model

In [4]:
# If you change the model, re-run all the cells below.
# Applicable models: wikiner_840B_300, wikiner_6B_300, wikiner_6B_100
MODEL_NAME = "wikiner_840B_300"

## 4. Some sample examples

In [5]:
# Enter examples to be transformed as strings in this list
text_list = [
              """William Henry Gates III (ur. 28 października 1955 r.) To amerykański magnat biznesowy, programista, inwestor i filantrop. Najbardziej znany jest jako współzałożyciel Microsoft Corporation. Podczas swojej kariery w Microsoft Gates zajmował stanowiska prezesa, dyrektora generalnego (CEO), prezesa i głównego architekta oprogramowania, będąc jednocześnie największym indywidualnym akcjonariuszem do maja 2014 r. Jest jednym z najbardziej znanych przedsiębiorców i pionierów rewolucja mikrokomputerowa lat 70. i 80. Urodzony i wychowany w Seattle w stanie Waszyngton, Gates był współzałożycielem Microsoftu z przyjacielem z dzieciństwa Paulem Allenem w 1975 r. W Albuquerque w Nowym Meksyku; stała się największą na świecie firmą produkującą oprogramowanie komputerowe. Gates prowadził firmę jako prezes i dyrektor generalny, aż do ustąpienia ze stanowiska dyrektora generalnego w styczniu 2000 r., Ale pozostał przewodniczącym i został głównym architektem oprogramowania. Pod koniec lat 90. Gates był krytykowany za taktykę biznesową, którą uważano za antykonkurencyjną. Opinię tę podtrzymują liczne orzeczenia sądowe. W czerwcu 2006 r. Gates ogłosił, że przejdzie do pracy w niepełnym wymiarze godzin w Microsoft i pracy w pełnym wymiarze godzin w Bill & Melinda Gates Foundation, prywatnej fundacji charytatywnej, którą on i jego żona Melinda Gates utworzyli w 2000 r. [ 9] Stopniowo przeniósł obowiązki na Raya Ozziego i Craiga Mundie. Zrezygnował z funkcji prezesa Microsoftu w lutym 2014 r. I objął nowe stanowisko jako doradca ds. Technologii, aby wesprzeć nowo mianowaną CEO Satyę Nadellę.""",
              """Mona Lisa to XVI-wieczny obraz olejny stworzony przez Leonarda. Odbywa się w Luwrze w Paryżu."""
            ]

## 5. Define Spark NLP pipeline

In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The wikiner_840B_300 is trained with glove_840B_300, so the embeddings in the
# pipeline should match. Same applies for the other available models.
if MODEL_NAME == "wikiner_840B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', lang='xx') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
elif MODEL_NAME == "wikiner_6B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_6B_300', lang='xx') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
elif MODEL_NAME == "wikiner_6B_100":
    embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')

ner_model = NerDLModel.pretrained(MODEL_NAME, 'pl') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            embeddings,
            ner_model,
            ner_converter])

glove_840B_300 download started this may take some time.
Approximate size to download 2.3 GB
[OK!]
wikiner_840B_300 download started this may take some time.
Approximate size to download 14.5 MB
[OK!]


## 6. Run the pipeline

In [9]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlp_pipeline.fit(df).transform(df)

## 7. Visualize results

In [10]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)